<a href="https://colab.research.google.com/github/fghinzelli/imersao-alura-google-ia/blob/main/Desafio_Imersao_Alura_Google_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install -U -q google-generativeai

In [49]:
# Importação de biblioteas e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
import requests

from bs4 import BeautifulSoup
from google.colab import userdata

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [10]:
# Funcao para buscar rotas do site SOS Enchentes
def getRoutes():
  url = 'https://sosenchentes.rs.gov.br/mapa-do-site'
  response = requests.get(url)
  routes = []

  if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    # Encontrar todos os links de páginas
    pages = paragrafos = soup.find_all('li', class_='mapa-site__1')

    for page in pages:
      routes.append(page.find('a').get('href'))

    return routes

In [29]:
# Acessa as páginas do SOS Enchentes e retorna o conteudo
def getContent():
  content = []
  for page in getRoutes():
    url = 'https://sosenchentes.rs.gov.br' + page
    response = requests.get(url)
    if response.status_code == 200:
      soup = BeautifulSoup(response.content, 'html.parser')

      cleaned_texts = []
      txt_content = soup.find_all('div', class_='artigo__texto')
      if (txt_content):
        for txt_item in txt_content:
          cleaned_texts.append(txt_item.get_text())

        page_content = {
            "titulo": soup.find('title').get_text(),
            "conteudo": ' '.join(cleaned_texts)
        }
        content.append(page_content)
  return content

documents = getContent()


In [30]:
# Cria o dataFrame com os dados das páginas
df = pd.DataFrame(documents)
df.columns = ["titulo", "conteudo"]
df

,titulo,conteudo
0,Avisos e Alertas - Defesa Civil do Rio Grande ...,\nEmbasada em diversos centros e institutos de...
1,Sobre o pix - SOS Rio Grande do Sul,\nDoações em dinheiro via pix\nO governo gaúch...
2,International donations - SOS Rio Grande do Sul,"\nThe State government, through the Banco do E..."
3,Como pedir socorro - SOS Rio Grande do Sul,\nEm caso de necessidade de resgate ou assistê...
4,Doações em geral - SOS Rio Grande do Sul,\nCentro Logístico da Defesa Civil EstadualAve...
5,Abrigos - SOS Rio Grande do Sul,\nABRIGAMENTO\nO abrigamento é de responsabili...
6,Situação nos municípios - SOS Rio Grande do Sul,\nDefesa Civil atualiza balanço das enchentes ...
7,"Água, luz e telefonia - SOS Rio Grande do Sul",\nAtualização dos serviços de infraestrutura d...
8,Retomada das aulas - SOS Rio Grande do Sul,"\nPanorama nas escolas estaduais - 10/5, 18h\n..."
9,Voluntários - SOS Rio Grande do Sul,\n\nOrientações sobre cadastro de voluntários\...


In [31]:
model = "models/embedding-001"

In [32]:
# Funcao para criação dos embedddings
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [33]:
# Inclusão dos embeddings no dataFrame
df["embeddings"] = df.apply(lambda row: embed_fn(row["titulo"], row["conteudo"]), axis=1)
df

,titulo,conteudo,embeddings
0,Avisos e Alertas - Defesa Civil do Rio Grande ...,\nEmbasada em diversos centros e institutos de...,"[0.010150316, -0.022668373, -0.006238854, 0.01..."
1,Sobre o pix - SOS Rio Grande do Sul,\nDoações em dinheiro via pix\nO governo gaúch...,"[0.03973951, -0.0033785582, -0.06536623, -0.00..."
2,International donations - SOS Rio Grande do Sul,"\nThe State government, through the Banco do E...","[0.03521929, -0.010249339, -0.057829753, -0.00..."
3,Como pedir socorro - SOS Rio Grande do Sul,\nEm caso de necessidade de resgate ou assistê...,"[0.011567781, -0.0061160885, -0.05875592, -0.0..."
4,Doações em geral - SOS Rio Grande do Sul,\nCentro Logístico da Defesa Civil EstadualAve...,"[0.021852612, -0.017239675, -0.058610436, -0.0..."
5,Abrigos - SOS Rio Grande do Sul,\nABRIGAMENTO\nO abrigamento é de responsabili...,"[0.024180492, -0.03226603, -0.05905637, 0.0230..."
6,Situação nos municípios - SOS Rio Grande do Sul,\nDefesa Civil atualiza balanço das enchentes ...,"[0.022411618, 0.010698517, -0.062454563, 0.009..."
7,"Água, luz e telefonia - SOS Rio Grande do Sul",\nAtualização dos serviços de infraestrutura d...,"[0.0110440655, -0.0060389293, -0.063338995, 0...."
8,Retomada das aulas - SOS Rio Grande do Sul,"\nPanorama nas escolas estaduais - 10/5, 18h\n...","[0.0061823563, -0.019295448, -0.05576219, 0.03..."
9,Voluntários - SOS Rio Grande do Sul,\n\nOrientações sobre cadastro de voluntários\...,"[0.033344276, -0.020965762, -0.06021302, -0.01..."


In [34]:
# Gera embeddings da consulta e busca o melhor resultado
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["conteudo"]

In [37]:
# Configuracoes do modelo
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [47]:
# Loop de iteração para perguntas
consulta = input('Faça sua pergunta: ')
while consulta != "fim":
  resultado = gerar_e_buscar_consulta(consulta, df, model)
  prompt = f"Resuma esse texto de uma forma simples e objetiva, sem adicionar informações que não façam parte do texto: {resultado}"
  model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
  response = model_2.generate_content(prompt)
  print("Resposta:", response.text, '\n\n')
  consulta = input('Faça sua pergunta: ')


Faça sua pergunta: O que posso doar?
Resposta: **Centro Logístico da Defesa Civil Estadual**

**Endereço:** Avenida Joaquim Porto Villanova, 101, Jardim Carvalho, Porto Alegre
**Telefone:** (51) 3210 4255

**Itens para Doação:**

* Colchões (novos ou em bom estado)
* Roupa de cama e banho
* Cobertores
* Água potável
* Ração animal
* Cestas básicas fechadas
* Fraldas infantis e geriátricas

**Itens da Cesta Básica:**

* Arroz
* Feijão
* Macarrão
* Açúcar
* Farinha de trigo
* Café
* Sal
* Óleo
* Biscoito
* Achocolatado
* Leite em pó
* Enlatados (atum, sardinha, legumes)

**Itens de Limpeza:**

* Luvas
* Botas de borracha
* Baldes
* Panos
* Vassouras (com cabo)
* Rodos (com cabo)
* Escova de limpeza

**Kits de Limpeza:**

* Água sanitária (1 litro)
* Desinfetante (1 litro)
* Sabão em pó (1 quilo)
* Detergente líquido (500 ml)
* Esponjas
* Panos de limpeza

**Kits de Higiene Pessoal:**

* Escovas de dente (2)
* Creme dental
* Sabonete
* Desodorante
* Shampoo
* Absorventes femininos
* Apare